In [5]:
# 예제 4-1 인스타그램 검색결과 URL을 만드는 함수
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

In [8]:
# 예제 4-2 selenium으로 URL 접속하기 - 1
from selenium import webdriver
driver = webdriver.Chrome('C:/Users/kang9/Desktop/인스타그램/chromedriver')

In [9]:
import time

In [10]:
url = "https://www.instagram.com/explore/tags/{}/".format(keyword)

In [11]:

# 예제 4-2 selenium으로 URL 접속하기 - 4
word = "을지로"
url = insta_searching(word)
driver.get(url)

In [12]:

# 예제 4-3 HTML에서 첫번째 게시글 찾아 클릭하기
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

In [13]:
# 예제 4-4 게시글 정보 가져오기
import re
from bs4 import BeautifulSoup
import unicodedata   # 추가, 2020.08.20

def get_content(driver):
    # ① 현재 페이지 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # ② 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
        content = unicodedata.normalize('NFC', content)   # 신규 추가. 2020.08.20
    except:
        content = ' '
    # ③ 본문 내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # ④ 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # ⑤ 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # ⑥ 위치정보 가져오기
    try: 
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)      # 신규 추가. 2020.08.20
    except:
        place = ''
    # ⑦ 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

['나랑 눈싸움 이기면 100만원', '2020-09-06', '129', '을지로', []]

In [14]:

# 예제 4-5 다음 게시글 열기
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

move_next(driver)

In [15]:


# ④ 첫 번째 게시글 열기
select_first(driver)

# ⑤ 비어있는 변수(results)만들기
results = [ ]


# ⑥→⑦→⑧ 여러 게시물 수집하기
target = 50      # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로)  2초 대기 후, 다음 게시글로 넘어가도록 try, except 구문 활용
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    

print(results[:2])

[['나랑 눈싸움 이기면 100만원', '2020-09-06', '129', '을지로', []], ['⠀저는 개강을했을수도있고 안했을수도있습니다.지원아 이제 서울여자라고 나버리지말아줘⠀', '2020-09-07', '687', '', []]]


In [17]:
# 예제 4-8 여러 개의 저장파일 통합하기
import pandas as pd
euljiro_insta_df = pd.DataFrame( [ ] )

folder = './files/'
f_list = ['3_1_crawling_jejudoMatJip.xlsx', '3_1_crawling_jejudoGwanGwang.xlsx', '3_1_crawling_jejuMatJip.xlsx', '3_1_crawling_jejuYeoHang.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns =['content','data','like','place','tags']

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

In [ ]:
jeju_insta_df.drop_duplicates(subset = [ "content"] , inplace = True)
jeju_insta_df.to_excel('./files/3_1_crawling_raw.xlsx', index = False)